API Documentation https://api.rawg.io/docs/  
API Key: 91bb4afb78e246729b0c89e7a6c5539a


In [1]:
import pandas as pd
import requests
import json
import rawgpy
from pathlib import Path

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=3)
    print(text)

savepath = Path("data/RAWG")

In [13]:
parameters = {
    "key": "91bb4afb78e246729b0c89e7a6c5539a",
    "page" : 1,
    "page_size":  40,
    #"search": "Call of Duty"
}

header = {'user-agent': 'Jonas Klenk'}


In [12]:
nextp = True
game_ids = set(())
games_list = pd.DataFrame()
page = parameters["page"]
afer_every_x_pages = 100

response = requests.get("https://api.rawg.io/api/games", params=parameters, headers = header)
print(response.json()["count"])

#Solange es noch weitere Seiten gibt:
while nextp == True :
    response_json = response.json()
    games = response_json["results"]
    sb = len(games)
    print(f"Page {page}: has {sb} Games")
    
    #Für jedes Spiel in den Resultaten:
    for game in games:
        #Falls das Game noch nicht da ist:
        if game["id"] not in game_ids:
            game_ids.add(game["id"])
            #Füge das Spiel in die Liste hinzu
            games_list = games_list.append(game, ignore_index=True)
    #Nach x pages
    if page % afer_every_x_pages == 0:
        #Speichere Dateien zwischen
        filecount = round(page / afer_every_x_pages)
        save_as = Path(f"{savepath}/games{filecount}.jsonl")
        games_list.to_json(save_as, orient="records", lines = True)
        print(f"File {save_as} is saved. Until page {page}")
        games_list = pd.DataFrame()
    
    page += 1

    # if page == 5:
    #     break

    if not response_json["next"]:
        nextp = False
        break
    response = requests.get(response_json["next"])


save_as = Path(f"{savepath}/gamesremainder.jsonl")
games_list.to_json(save_as, orient="records", lines = True)
print(f"File {save_as} is saved. Until page {page}")


459174
Page 1: has 40 Games
Page 2: has 40 Games
Page 3: has 14 Games


MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?